In [1]:
import io
import PIL
from PIL import Image
import requests
import time
import selenium
from selenium import webdriver
import hashlib

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=2):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);") #executescript method executes JavaScript in the context of the currently selected frame
        #in Selenium. 
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [2]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content #requests.get() sends a GET request to a specified url. .content is usesd when the server returns binary data

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content) #in memory binary stream
        image = Image.open(image_file).convert('RGB') #returns a converted copy of this image
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [3]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=100):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [7]:
search_term = 'Joe Biden face'
DRIVER_PATH = "/usr/local/bin/chromedriver" #chrome driver is installed here!!! from brew install chromedriver

search_and_download(
    search_term = search_term,
    driver_path=DRIVER_PATH
)

/var/folders/fc/9dptd99s0h5f47szq92yhync0000gn/T/ipykernel_11885/640344845.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome(executable_path=driver_path) as wd:
/var/folders/fc/9dptd99s0h5f47szq92yhync0000gn/T/ipykernel_11885/548976680.py:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


/var/folders/fc/9dptd99s0h5f47szq92yhync0000gn/T/ipykernel_11885/548976680.py:43: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 101 image links, done!
SUCCESS - saved https://cloudfront-us-east-2.images.arcpublishing.com/reuters/QO3W4XBFGJM5TIBA7Q5ESZY3XQ.jpg - as ./images/joe_biden_face/2e46d3fa41.jpg
SUCCESS - saved https://www.washingtonpost.com/politics/interactive/2021/biden-fact-checker-100-days/img/JoeBiden_tophalf.png - as ./images/joe_biden_face/b9bb24cfcf.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQfsodQuZVzXdYTbaamwd8ixtkakSH2OUiGKg&usqp=CAU - as ./images/joe_biden_face/d1a188fe11.jpg
SUCCESS - saved https://ktla.com/wp-content/uploads/sites/4/2021/11/President-Biden.jpg?w=1280 - as ./images/joe_biden_face/beaca49528.jpg
SUCCESS - saved https://cloudfront-us-east-2.images.arcpublishing.com/reuters/QES3ISDRWFPMDGNEZRRTFLNWZE.jpg - as ./images/joe_biden_face/7e63e43d08.jpg
SUCCESS - saved https://www.sun-sentinel.com/resizer/PbY_uWvM03QKI0jc_OifceWOB40=/415x276/top/cloudfront-us-east-1.images.arcpublishing.com/tronc/WHD22VG7PVRQP6TK3GCZDJ7DJE.aspx - as ./images/joe